In [31]:
import os
import pandas as pd
import numpy as np

os.listdir()

['.ipynb_checkpoints',
 '01_Data_loading.ipynb',
 '02_Data_pre-processing.ipynb',
 '03_data_out.ipynb',
 'BTCUSDT.csv',
 'BTCUSDT_210701_220630.csv',
 'BTCUSDT_220705.csv',
 'bybit.key',
 'bybit.txt',
 'lstm.h5',
 'out_BTCUSDT_220705.csv',
 'temp.csv',
 'vae_encoder.h5']

In [34]:
FILENAME='BTCUSDT.csv'

In [35]:
df = pd.read_csv(FILENAME)

In [36]:
df

,open_time,open,high,close,low,volume
0,2022-06-01 00:00:00,31781.0,31804.0,31804.0,31780.5,91.441
1,2022-06-01 00:01:00,31804.0,31828.0,31824.5,31804.0,36.907
2,2022-06-01 00:02:00,31824.5,31835.0,31833.0,31813.0,55.765
3,2022-06-01 00:03:00,31833.0,31833.5,31825.0,31825.0,43.427
4,2022-06-01 00:04:00,31825.0,31825.5,31825.0,31825.0,6.091
...,...,...,...,...,...,...
43195,2022-06-30 23:55:00,19824.5,19838.0,19737.0,19727.0,1086.056
43196,2022-06-30 23:56:00,19737.0,19822.0,19819.5,19723.5,829.685
43197,2022-06-30 23:57:00,19819.5,19820.0,19799.5,19767.0,526.045
43198,2022-06-30 23:58:00,19799.5,19866.0,19853.5,19767.0,871.622


In [37]:
df.columns = ["Time","Open","High","Low","Close","Volume"]

In [38]:
df

,Time,Open,High,Low,Close,Volume
0,2022-06-01 00:00:00,31781.0,31804.0,31804.0,31780.5,91.441
1,2022-06-01 00:01:00,31804.0,31828.0,31824.5,31804.0,36.907
2,2022-06-01 00:02:00,31824.5,31835.0,31833.0,31813.0,55.765
3,2022-06-01 00:03:00,31833.0,31833.5,31825.0,31825.0,43.427
4,2022-06-01 00:04:00,31825.0,31825.5,31825.0,31825.0,6.091
...,...,...,...,...,...,...
43195,2022-06-30 23:55:00,19824.5,19838.0,19737.0,19727.0,1086.056
43196,2022-06-30 23:56:00,19737.0,19822.0,19819.5,19723.5,829.685
43197,2022-06-30 23:57:00,19819.5,19820.0,19799.5,19767.0,526.045
43198,2022-06-30 23:58:00,19799.5,19866.0,19853.5,19767.0,871.622


In [39]:
def preprocess_ratio(data):
    windows = [5,10,20,60,120]
    for window in windows:
        data['close_ma{}'.format(window)] =  data['Close'].rolling(window).mean()
        data['volume_ma{}'.format(window)] = data['Volume'].rolling(window).mean()
        data['close_ema{}'.format(window)] =  data['Close'].ewm(span=window).mean()
        data['volume_ema{}'.format(window)] = data['Volume'].ewm(span=window).mean()
        data['close_ma%d_ratio' % window] = (data['Close'] - data['close_ma%d' % window]) / data['close_ma%d' % window]
        data['volume_ma%d_ratio' % window] = (data['Volume'] - data['volume_ma%d' % window]) / data['volume_ma%d' % window]
        data['close_ema%d_ratio' % window] = (data['Close'] - data['close_ema%d' % window]) / data['close_ema%d' % window]
        data['volume_ema%d_ratio' % window] = (data['Volume'] - data['volume_ema%d' % window]) / data['volume_ema%d' % window]
        
    data['open_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'open_lastclose_ratio'] = (data['Open'][1:].values - data['Close'][:-1].values) / data['Close'][:-1].values
    
    data['high_close_ratio'] = (data['High'].values - data['Close'].values) / data['Close'].values
    
    data['low_close_ratio'] = (data['Low'].values - data['Close'].values) / data['Close'].values
    
    data['close_lastclose_ratio'] = np.zeros(len(data))
    data.loc[1:, 'close_lastclose_ratio'] = (data['Close'][1:].values - data['Close'][:-1].values) / data['Close'][:-1].values
    
    data['volume_lastvolume_ratio'] = np.zeros(len(data))
    data.loc[1:, 'volume_lastvolume_ratio'] = (data['Volume'][1:].values - data['Volume'][:-1].values) / data['Volume'][:-1].replace(to_replace=0, method='ffill').replace(to_replace=0, method='bfill').values
    
    shortema = data['Close'].ewm(span=12).mean()
    longema = data['Close'].ewm(span=26).mean()
    DIF = shortema - longema
    DEA = DIF.ewm(span=9).mean()
    data['MACD'] = 2*(DIF-DEA)
    
    delta = data['Close'].diff(1)
    delta = delta[1:]
    
    up = delta.copy()
    down = delta.copy()
    up[up<0] = 0
    down[down>0] = 0
    data['up'] = up
    data['down'] = down
    
    avg_gain = data['up'].rolling(window=14).mean()
    avg_loss = abs(data['down'].rolling(window=14).mean())
    RS = avg_gain / avg_loss
    
    RSI = 100.0-(100.0/(1.0+RS))
    data['RSI'] = RSI
    
    ndays_high=data["High"].rolling(window=14, min_periods=1).max()
    ndays_low=data['Low'].rolling(window=14, min_periods=1).min()
    fast_k = ((data["Close"]-ndays_low)/(ndays_high-ndays_low))*100
    data["Fast_k"]=fast_k
    
    typical_price = ((df['Close'] + df['High'] + df['Low']) / 3)
    money_flow = typical_price * df['Volume']
    positive_flow = []
    negative_flow = []
    for i in range(1, len(typical_price)):
        if typical_price[i] > typical_price[i-1] :
            positive_flow.append(money_flow[i-1])
            negative_flow.append(0)
        elif typical_price[i] < typical_price[i-1]:
            negative_flow.append(money_flow[i-1])
            positive_flow.append(0)
        else:
            positive_flow.append(0)
            negative_flow.append(0)
            
    MF_period = 14
    positive_mf = [np.nan]*14
    negative_mf = [np.nan]*14
    for i in range(MF_period-1, len(positive_flow)):
        positive_mf.append(sum(positive_flow[i+1-MF_period : i+1]))
        
    for i in range(MF_period-1, len(negative_flow)):
        negative_mf.append(sum(negative_flow[i+1-MF_period : i+1]))
        
    mfi = 100 * (np.array(positive_mf) / (np.array(positive_mf) + np.array(negative_mf)))
    df['MFI'] = mfi
    
    data=data.drop(columns=['up','down'])
    data=data.dropna(axis=0)
    
    return data

In [40]:
temp=preprocess_ratio(df)

In [41]:
temp

,Time,Open,High,Low,Close,Volume,close_ma5,volume_ma5,close_ema5,volume_ema5,...,volume_ema120_ratio,open_lastclose_ratio,high_close_ratio,low_close_ratio,close_lastclose_ratio,volume_lastvolume_ratio,MACD,RSI,Fast_k,MFI
119,2022-06-01 01:59:00,31910.0,31919.5,31919.0,31910.0,81.209,31898.2,79.6376,31896.188831,74.781966,...,0.754098,0.000219,0.000298,0.000282,0.000219,-0.022320,20.000756,94.711538,90.776699,90.817700
120,2022-06-01 02:00:00,31919.0,31919.5,31905.5,31900.0,95.428,31901.6,77.4962,31897.459221,81.663977,...,1.020347,0.000282,0.000611,0.000172,-0.000313,0.175091,17.810787,86.403509,81.067961,82.007609
121,2022-06-01 02:01:00,31905.5,31912.0,31912.0,31905.0,37.872,31904.1,64.9128,31899.972814,67.066652,...,-0.195162,0.000172,0.000219,0.000219,0.000157,-0.603135,16.081151,86.462882,85.922330,83.652088
122,2022-06-01 02:02:00,31912.0,31912.0,31912.0,31911.5,17.175,31905.9,62.9494,31903.815209,50.436101,...,-0.630554,0.000219,0.000016,0.000016,0.000204,-0.546499,14.885505,91.341991,92.233010,86.144146
123,2022-06-01 02:03:00,31912.0,31912.0,31911.5,31911.5,21.722,31907.6,50.6812,31906.376806,40.864734,...,-0.527987,0.000016,0.000016,0.000000,0.000000,0.264745,13.223735,91.341991,90.588235,89.326028
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43195,2022-06-30 23:55:00,19824.5,19838.0,19737.0,19727.0,1086.056,19678.2,2121.7998,19652.813189,1567.749226,...,1.340118,0.002985,0.005627,0.000507,-0.001948,0.029191,125.044457,94.973890,76.741573,65.146818
43196,2022-06-30 23:56:00,19737.0,19822.0,19819.5,19723.5,829.685,19712.6,1068.5456,19676.375459,1321.727817,...,0.764740,0.000507,0.004994,0.004867,-0.000177,-0.236057,110.668263,94.499018,76.348315,66.686559
43197,2022-06-30 23:57:00,19819.5,19820.0,19799.5,19767.0,526.045,19731.6,942.1756,19706.583640,1056.500212,...,0.116703,0.004867,0.002681,0.001644,0.002205,-0.365970,98.984767,94.795539,80.979499,68.294412
43198,2022-06-30 23:58:00,19799.5,19866.0,19853.5,19767.0,871.622,19750.0,873.7320,19726.722426,994.874141,...,0.824659,0.001644,0.005008,0.004376,0.000000,0.656934,83.823729,94.795539,79.496624,68.984831


In [25]:
temp.to_csv(FILENAME, mode='w',index=False)

In [26]:
temp.head(2)

,Time,Open,High,Low,Close,Volume,close_ma5,volume_ma5,close_ema5,volume_ema5,...,volume_ema120_ratio,open_lastclose_ratio,high_close_ratio,low_close_ratio,close_lastclose_ratio,volume_lastvolume_ratio,MACD,RSI,Fast_k,MFI
119,2022-06-01 01:59:00,31910.0,31919.5,31919.0,31910.0,81.209,31898.2,79.6376,31896.188831,74.781966,...,0.754098,0.000219,0.000298,0.000282,0.000219,-0.022320,20.000756,94.711538,90.776699,90.817700
120,2022-06-01 02:00:00,31919.0,31919.5,31905.5,31900.0,95.428,31901.6,77.4962,31897.459221,81.663977,...,1.020347,0.000282,0.000611,0.000172,-0.000313,0.175091,17.810787,86.403509,81.067961,82.007609
